In [8]:
import numpy as np
from numpy import linalg as LA
from tensornetwork import ncon
import logging

from scipy.sparse.linalg import LinearOperator, eigs

import math 
import copy 


In [14]:
d = 2
chi = 10

A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)

sAB = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights
sBA = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights

sigBA = np.random.rand(chi, chi)  # initialize random starting point

tol = 1e-14  # set convergence tolerance


# define the tensor network
tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
           np.diag(sAB), B, B.conj()]
labels = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]

for k in range(1000):
  sigBA_new = ncon([sigBA, *tensors], labels)  # contract transfer operator
  sigBA_new = sigBA_new / np.trace(sigBA_new)  # normalize
  if LA.norm(sigBA - sigBA_new) < tol:  # check convergence
    print('success!')
    break
  else:
    print('iter: %d, diff: %e' % (k, LA.norm(sigBA - sigBA_new)))
    # print(sigBA_new)
    sigBA = sigBA_new

sigBAv1 = copy.copy(sigBA)


sigBA = np.random.rand(chi, chi)

# initialize the starting vector
chiBA = A.shape[0]
if sigBA.shape[0] == chiBA:
    v0 = sigBA.reshape(np.prod(sigBA.shape))
else:
    v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

# define network for transfer operator contract
tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
            np.diag(sAB), B, B.conj()]
labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
        [8, 10, -1], [9, 10, -2]]

# define function for boundary contraction and pass to eigs
def left_iter(sigBA):
    return ncon([sigBA.reshape([chiBA, chiBA]), *tensors],
            labels).reshape([chiBA**2, 1])

Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                    k=1, which='LM', tol=tol)

# normalize the environment density matrix sigBA
if np.isrealobj(A):
    sigBA = np.real(sigBA)
sigBA = sigBA.reshape(chiBA, chiBA)
sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
sigBA = sigBA / np.trace(sigBA)

print(LA.norm(sigBA.reshape(chiBA,chiBA)-sigBAv1))


iter: 0, diff: 5.009089e+00
iter: 1, diff: 3.900482e-03
iter: 2, diff: 4.480649e-05
iter: 3, diff: 5.857921e-07
iter: 4, diff: 1.058770e-08
iter: 5, diff: 1.158899e-10
iter: 6, diff: 1.444826e-12
iter: 7, diff: 1.829284e-14
success!
4.873060251453353e-16
